# 星宸药研 | DeepChem中文教程2.
# 使用DeepChem预测PROTAC降解剂的降解活性

DeepChem是一个专门用于化学、材料科学和生物学的机器学习库，支持多种数据类型和模型结构。
在本教程中，将展示如何使用DeepChem来训练一个简单的机器学习模型。


DeepChem GitHub仓库: [https://github.com/deepchem/deepchem](https://github.com/deepchem/deepchem)

星宸药研 GitHub仓库: [https://github.com/stardrug-c01/Deepchem-Chinese](https://github.com/stardrug-c01/Deepchem-Chinese)

本中文教程将保存于星宸药研 GitHub以及星宸药研微信公众号。

## 前情提要

在使用本教程之前，你需要先安装DeepChem的环境。请参考以下链接进行安装：
[如何安装DeepChem环境](https://mp.weixin.qq.com/s/dDyKw7LFTtFKJyyHt54xmw)

这是小编使用的conda环境概述:

## 关键依赖项 (Dependencies)
- **cudatoolkit**: `11.3`     
- **cudnn**: `8.9.2.26`
- **Python**: `3.9.19`
- **DeepChem**: `2.5.0`
- **TensorFlow**: `2.10.1` 
- **PyTorch**: `1.12.1` (CUDA 11.3)   #没有显卡可到pytorch官网找cpu版本的下载命令
- **TorchDrug**: `0.2.1`
- **RDKit**: `2024.03.5`
- **Keras**: `3.4.1`
- **NumPy**: `1.26.4`
- **Pandas**: `2.2.2`
- **Scikit-learn**: `1.5.1`
- **SciPy**: `1.13.1`
- **MATPLOTLIB**: `3.9.1`

## 其他依赖项 (Other Dependencies)

- **CFFI**: `1.16.0`
- **IPython**: `8.18.1`
- **JupyterLab**: `4.2.4`
- **Bokeh**: `3.3.2`
- **TQDM**: `4.66.4`
- **Tornado**: `6.4.1`
- **protobuf**: `4.25.4`
- **GRPCIO**: `1.65.1`
- **Tensorboard**: `2.17.0`

## 通过 pip 安装的库

- **absl-py**: `2.1.0`
- **flatbuffers**: `24.3.25`
- **opt-einsum**: `3.3.0`
- **markdown-it-py**: `3.0.0`
- **tensorflow-io-gcs-filesystem**: `0.37.1`
- **werkzeug**: `3.0.3`

---


In [ ]:
# 确认Deepchem已安装

import deepchem as dc
dc.__version__

首先我们需要先导入需要的库和下载数据集(后续也可以替换成自己的数据集试试噢，但是请记得格式问题)

In [2]:
import os  # 用于与操作系统交互，例如文件和目录管理

import deepchem as dc     # DeepChem库，用于化学数据处理和机器学习
import rdkit              # RDKit库，用于化学信息学和分子操作
from rdkit import Chem    # RDKit中的Chem模块，用于化学分子操作
from rdkit.Chem import Draw  # RDKit中的Draw模块，用于绘制化学结构

import pandas as pd  # pandas库，用于数据处理和分析
import numpy as np  # numpy库，用于数值计算和数组操作
import matplotlib.pyplot as plt  # matplotlib库，用于数据可视化

In [ ]:
os.system('wget https://deepchemdata.s3.us-west-1.amazonaws.com/datasets/protac_10_06_24.csv')
# 使用系统命令下载数据文件
# 'wget' 是一个命令行工具，用于从网络上下载文件
# 'https://deepchemdata.s3.us-west-1.amazonaws.com/datasets/protac_10_06_24.csv' 是要下载的数据文件的URL

In [4]:
protac_db = pd.read_csv('protac_10_06_24.csv')
# 使用pandas库的read_csv函数读取CSV文件
# 'protac_10_06_24.csv' 是要读取的文件名
# 读取的内容将存储在变量protac_db中，通常以DataFrame的形式存储



请注意，PROTAC化合物和靶标蛋白之间存在多对多的映射关系。一个PROTAC化合物可以设计成靶向多个蛋白质，反之，多个PROTAC化合物也可以开发来靶向同一个蛋白质。这种多对多的关系为PROTACs的设计和应用提供了更大的灵活性和适应性。

In [ ]:
print('''In this dataset, there are {} unique PROTAC compounds, targeting {} unique proteins for a total of {} combinations'''.format(len(protac_db['Compound ID'].unique()),
                                                                                   len(protac_db['Target'].unique()), protac_db.shape[0]))
# 打印数据集的信息
# 使用字符串格式化方法，插入数据集中唯一的PROTAC化合物数量、唯一的靶标蛋白数量以及总的组合数
# len(protac_db['Compound ID'].unique()) 计算唯一的PROTAC化合物数量
# len(protac_db['Target'].unique()) 计算唯一的靶标蛋白数量
# protac_db.shape[0] 计算数据集中的总行数，即组合数
protac_db
# 显示数据集内容
#在这个数据集中，共有3270种独特的PROTAC化合物，靶向323种独特的蛋白质，总共有5388种组合。

In [ ]:
example = protac_db.iloc[0]
# 从数据集中选择第一行作为示例
# iloc[0] 获取数据集的第一行数据

print('''Here is the SMILEs representation of a PROTAC compound: {}
designed to target {} protein through ubiquitination by {} E3 ligase.'''.format(example['Smiles'], example['Target'], example['E3 ligase']))
# 打印PROTAC化合物的SMILES表示形式、靶标蛋白以及E3泛素连接酶的信息
# example['Smiles'] 获取示例化合物的SMILES字符串
# example['Target'] 获取示例化合物靶向的蛋白质
# example['E3 ligase'] 获取示例化合物通过E3泛素连接酶进行泛素化的信息


In [ ]:
protac_db.columns
# 显示数据集的所有列名
# 这有助于了解数据集中包含了哪些信息



一般而言，PROTAC-DB数据集包含了PROTAC结构的各种生理化学和生物化学性质的信息。其中几个有用的指标包括：

ΔG（自由能变化），描述了化学反应的自发性。
Kd（解离常数），衡量配体达到蛋白质结合位点50%占据所需的浓度。
XLogP3，衡量化合物的疏水性，指示其溶解性以及吸收和分布特性。
在继续之前，让我们绘制这些属性的分布图，以更好地了解我们的PROTAC数据集，从ΔG值开始。

In [ ]:
delta_G = protac_db['delta G (kcal/mol, Protac to E3)']
# 提取数据集中'ΔG'列的数据

delta_G = delta_G.dropna()
# 删除包含缺失值的数据行

delta_G = delta_G.astype(float)
# 将数据转换为浮点型，以确保可以进行数值计算

plt.hist(delta_G, bins=10)
# 绘制直方图，显示ΔG值的分布
# bins=10 表示将数据分成10个区间

plt.xlabel('ΔG (kcal/mol)')
# 设置x轴标签为 'ΔG (kcal/mol)'

plt.ylabel('Frequency')
# 设置y轴标签为 'Frequency'

plt.title(f'Distribution of ΔG across PROTAC molecules')
# 设置图表标题为 'ΔG在PROTAC分子中的分布'

plt.plot()
# 显示图表


让我们仔细看看ΔG值在-10范围内的PROTAC分子分布情况。

In [ ]:
delta_G = protac_db['delta G (kcal/mol, Protac to E3)']
# 提取数据集中'ΔG'列的数据

delta_G = delta_G.dropna()
# 删除包含缺失值的数据行

delta_G = delta_G.astype(float)
# 将数据转换为浮点型，以确保可以进行数值计算

x_min = -15
x_max = -5
bin_size = 1
bins = np.arange(x_min, x_max, bin_size)
# 设置x轴范围（-15到-5）和每个区间的大小（1）
# 使用np.arange生成区间边界

plt.hist(delta_G, bins=bins)
# 绘制直方图，显示ΔG值在-15到-5范围内的分布

plt.xlabel('ΔG (kcal/mol)')
# 设置x轴标签为 'ΔG (kcal/mol)'

plt.ylabel('Frequency')
# 设置y轴标签为 'Frequency'

plt.title('Distribution of ΔG ranged from -15 to -5 across PROTAC molecules')
# 设置图表标题为 'ΔG在-15到-5范围内的PROTAC分子分布'

plt.plot()
# 显示图表


目前，PROTAC反应的自发性信息似乎不多，但值得注意的是，记录的ΔG值看起来都符合预期，显示出反应在能量上是有利的。

接下来，我们来看看Kd值。

In [ ]:
kd_data = protac_db['Kd (nM, Ternary complex)']
# 提取数据集中'Kd'列的数据

kd_data = kd_data.dropna()
# 删除包含缺失值的数据行

kd_data = kd_data[~kd_data.str.contains('/')]
# 移除包含斜杠('/')的值，这些值可能表示数据记录中的异常或错误

kd_data = kd_data.astype(float)
# 将数据转换为浮点型，以确保可以进行数值计算

plt.hist(kd_data)
# 绘制直方图，显示Kd值的分布

plt.xlabel('Dissociation constant (nM)')
# 设置x轴标签为 '解离常数 (nM)'

plt.ylabel('Frequency')
# 设置y轴标签为 'Frequency'

plt.title('Distribution of Kd values across PROTAC molecules')
# 设置图表标题为 'Kd值在PROTAC分子中的分布'

plt.plot()
# 显示图表


与ΔG值类似，PROTAC复合物的亲和力信息似乎也不多。由于Kd值的范围很大，让我们绘制一个聚焦于低Kd值的PROTACs的直方图。

In [ ]:
kd_data = protac_db['Kd (nM, Ternary complex)']
# 提取数据集中'Kd'列的数据

kd_data = kd_data.dropna()
# 删除包含缺失值的数据行

kd_data = kd_data[~kd_data.str.contains('/')]
# 移除包含斜杠('/')的值，这些值可能表示数据记录中的异常或错误

kd_data = kd_data.astype(float)
# 将数据转换为浮点型，以确保可以进行数值计算

# 限制范围
x_max = 1500
x_min = 0
bin_size = 25
bins = np.arange(x_min, x_max, bin_size)
# 设置x轴范围（0到1500）和每个区间的大小（25）
# 使用np.arange生成区间边界

plt.hist(kd_data, bins=bins)
# 绘制直方图，显示Kd值在0到1500范围内的分布

plt.xlabel('Dissociation constant (nM)')
# 设置x轴标签为 '解离常数 (nM)'

plt.ylabel('Frequency')
# 设置y轴标签为 'Frequency'

plt.title('Distribution of Kd values ranged from 0-1500 across PROTAC molecules')
# 设置图表标题为 'Kd值在0-1500范围内的PROTAC分子分布'

plt.plot()
# 显示图表


改进后的分辨率显示了Kd值的更清晰的分布。我们可以看到，一些Kd值较低，这表明PROTAC连接体可以与E3泛素连接酶和靶标蛋白形成强有力的结合。

接下来，让我们查看XLogP3值。请注意，这与典型的LogP分配系数稍有不同。回顾一下，LogP定义为：
LogP的定义是：

$$
\text{LogP} = \log \left( \frac{[\text{solute}]_{\text{oct}}}{[\text{solute}]_{\text{H}_2\text{O}}} \right)
$$


换句话说，LogP是化合物在有机相和水相中的浓度比，测量了化合物的溶解性。XLogP3 是一种基于知识的方法，用于计算分配系数，它考虑了分子结构、功能团的存在以及键合情况。这两种性质都估算了化合物的疏水性，提供了化合物在生物系统中可能的行为的见解。

In [ ]:
plt.hist(protac_db['XLogP3'])
# 绘制直方图，显示XLogP3值的分布

plt.xlabel('XLogP3 Values')
# 设置x轴标签为 'XLogP3值'

plt.ylabel('Frequency')
# 设置y轴标签为 'Frequency'

plt.title('Distribution of XLogP3 values across PROTAC molecules')
# 设置图表标题为 'XLogP3值在PROTAC分子中的分布'

plt.plot()
# 显示图表


所有的PROTAC化合物都有记录的XLogP3值。分布看起来近似正态分布，只有少数分子具有极端的LogP特征。

现在，让我们查看PROTAC降解性质。**"DC50 (nM)"和"Dmax (%)"**分别表示半最大降解浓度和目标蛋白质的最大降解百分比。让我们先快速查看它们的分布情况。

首先进行一些数据清理。

In [13]:
# 首先删除所有的NaN值
raw_dc50 = protac_db['DC50 (nM)']
# 提取数据集中'DC50 (nM)'列的数据

raw_dc50 = raw_dc50.dropna()
# 删除包含缺失值的数据行


请注意，这些值都以字符串格式存在，并且包含非数字字符，如<、/和>。暂时，我们先移除这些值。

In [14]:
raw_dc50 = raw_dc50[~raw_dc50.str.contains('<|>|/|~|-')]
# 移除包含字符 '<'、'>'、'/'、'~' 或 '-' 的值，这些字符可能表示数据记录中的异常或错误

raw_dc50 = raw_dc50.astype(float)
# 将剩余数据转换为浮点型，以确保可以进行数值计算


In [ ]:
plt.hist(raw_dc50.values, bins=75)
# 绘制直方图，显示DC50值的分布
# bins=75 表示将数据分成75个区间

plt.xlabel('PROTACs')
# 设置x轴标签为 'PROTACs'

plt.ylabel('DC50 (nM)')
# 设置y轴标签为 'DC50 (nM)'

plt.title('DC50 for all PROTACs')
# 设置图表标题为 '所有PROTACs的DC50值'

plt.plot()
# 显示图表



分布确实存在偏斜，并且有一些离群值。让我们进行对数归一化处理。

In [16]:
lognorm_dc50 = np.log(raw_dc50)
# 对DC50值进行自然对数变换，以进行对数归一化


In [ ]:
plt.hist(lognorm_dc50, bins=15)
# 绘制直方图，显示对数归一化后的DC50值的分布
# bins=15 表示将数据分成15个区间

plt.xlabel('Log normalized DC50 values (log nM)')
# 设置x轴标签为 '对数归一化的DC50值（对数 nM）'

plt.ylabel('Frequency')
# 设置y轴标签为 'Frequency'

plt.title('Distribution of log normalized DC50 values')
# 设置图表标题为 '对数归一化DC50值的分布'

plt.plot()
# 显示图表


现在，让我们查看Dmax百分比，它代表了PROTAC可以引起的最大降解程度，相对于目标蛋白质的总活性。

In [18]:
# 使用与清理后的DC50数据相同的行索引
dmax = protac_db.iloc[lognorm_dc50.index]['Dmax (%)']
# 从数据集中提取'Dmax (%)'列的数据，行索引与清理后的DC50数据相同


In [19]:
# 遵循相同的数据清理过程
dmax = dmax.dropna()
# 删除包含缺失值的数据行

dmax = dmax[~dmax.str.contains('<|>|/|~|-')]
# 移除包含字符 '<'、'>'、'/'、'~' 或 '-' 的值，这些字符可能表示数据记录中的异常或错误

dmax = dmax.astype(float)
# 将剩余数据转换为浮点型，以确保可以进行数值计算


In [ ]:
plt.hist(dmax.values, bins=10)
# 绘制直方图，显示Dmax (%)值的分布
# bins=10 表示将数据分成10个区间

plt.xlabel('Dmax (%)')
# 设置x轴标签为 'Dmax (%)'

plt.ylabel('Frequency')
# 设置y轴标签为 'Frequency'

plt.title('Distribution of Dmax (%)')
# 设置图表标题为 'Dmax (%)的分布'

plt.plot()
# 显示图表



请注意，Dmax以百分比表示。现在，我们继续对DC50进行回归分析。我们已经准备好进行特征提取了！

In [ ]:
# 目前我们要预测DC50属性
cleaned_data = protac_db.iloc[lognorm_dc50.index]
# 从原始数据集中提取与清理后的DC50数据相对应的行

print('There are {} PROTAC samples.'.format(cleaned_data.shape[0]))
# 打印PROTAC样本的数量


In [22]:
protac_smiles = cleaned_data['Smiles']
# 提取清理后的数据集中 'Smiles' 列的数据

dc_vals = lognorm_dc50
# 将对数归一化后的DC50值赋值给dc_vals


特征提取

让我们使用 DeepChem 中的 CircularFingerprint 进行特征提取！CircularFingerprint 是一种常用的分子特征提取方法，它编码了关于每个原子及其邻域的局部信息。

In [23]:
from rdkit import Chem
# 导入 RDKit 库中的 Chem 模块

featurizer = dc.feat.CircularFingerprint(radius=4, chiral=True)
# 初始化 CircularFingerprint 特征提取器，设置半径为4，并启用立体化学特征

features = featurizer.featurize(protac_smiles)
# 使用特征提取器对PROTAC的SMILES表示进行特征提取

# 初始化数据集并进行分割
dataset = dc.data.NumpyDataset(X=features, y=dc_vals, ids=list(protac_smiles))
# 创建一个 NumpyDataset 数据集，其中 X 是特征，y 是对数归一化后的 DC50 值，ids 是样本 ID

splitter = dc.splits.RandomSplitter()
# 初始化一个随机分割器

train_random, val_random, test_random = splitter.train_valid_test_split(dataset, seed=42)
# 使用随机分割器将数据集分为训练集、验证集和测试集，设置随机种子为42以保证结果可重复


除了随机分割，我们还将使用Scaffold Split，以确保数据分割包含结构多样的化合物。Scaffold Split 根据分子中环、连接子、环和连接子的组合，以及原子属性对分子进行分组。一般来说，Scaffold Split 是确保模型具有良好泛化能力的一个有效方法。

In [24]:
# Scaffold 分割
splitter = dc.splits.ScaffoldSplitter()
# 初始化一个 ScaffoldSplitter 对象

train_scaffold, val_scaffold, test_scaffold = splitter.train_valid_test_split(dataset, seed=42)
# 使用 ScaffoldSplitter 将数据集分为训练集、验证集和测试集，设置随机种子为42以保证结果可重复


为了查看 Scaffold Split 的效果，让我们可视化分割后的化合物。

In [ ]:
print("训练集中的三种分子：")
# 打印训练集中选择的三种分子

np.random.seed(42)
# 设置随机种子为42，以确保结果可重复

indices = np.random.choice(len(train_scaffold), size=3, replace=False)
# 从训练集中随机选择3个样本的索引

smiles = [train_scaffold.ids[index] for index in indices]
# 获取这3个样本的SMILES表示

mols = [Chem.MolFromSmiles(smile) for smile in smiles]
# 将SMILES表示转换为RDKit分子对象

Draw.MolsToGridImage(mols, molsPerRow=3, subImgSize=(450, 350))
# 将这些分子可视化为网格图像，每行显示3个分子，子图像大小为450x350像素


In [ ]:
print("验证集中的三种分子：")
# 打印验证集中选择的三种分子

indices = np.random.choice(len(val_scaffold), size=3, replace=False)
# 从验证集中随机选择3个样本的索引

smiles = [val_scaffold.ids[index] for index in indices]
# 获取这3个样本的SMILES表示

mols = [Chem.MolFromSmiles(smile) for smile in smiles]
# 将SMILES表示转换为RDKit分子对象

Draw.MolsToGridImage(mols, molsPerRow=3, subImgSize=(450, 350))
# 将这些分子可视化为网格图像，每行显示3个分子，子图像大小为450x350像素


In [ ]:
print("测试集中的三种分子：")
# 打印测试集中选择的三种分子

indices = np.random.choice(len(test_scaffold), size=3, replace=False)
# 从测试集中随机选择3个样本的索引

smiles = [test_scaffold.ids[index] for index in indices]
# 获取这3个样本的SMILES表示

mols = [Chem.MolFromSmiles(smile) for smile in smiles]
# 将SMILES表示转换为RDKit分子对象

Draw.MolsToGridImage(mols, molsPerRow=3, subImgSize=(450, 350))
# 将这些分子可视化为网格图像，每行显示3个分子，子图像大小为450x350像素


各数据集分割中确实存在功能团的差异。例如，训练集中出现了腈基，验证集中出现了胺基，而测试集中则有磺胺基。此外，不同数据分割中的结构和构象差异也很明显。这将有助于我们了解模型的泛化能力。

# 模型部署
我们已经成功生成了训练和测试数据集。现在让我们创建一个简单的多层感知机（MLP）模型来预测PROTAC的降解特性吧！

In [ ]:
import deepchem as dc
import tensorflow as tf

# 定义任务数和特征数
n_tasks = 1
n_features = train_random.X.shape[1]

# 定义每层的神经元数量
layer_sizes = [256, 32, 1]
# 定义每层的dropout率
dropouts = [0.0, 0.2, 0]
# 使用 TensorFlow 的激活函数
activation_fns = [tf.nn.relu, tf.nn.relu, tf.keras.activations.linear]

# 使用 DeepChem 的 Adam 优化器
optimizer = dc.models.optimizers.Adam()

# 记录每个训练epoch的日志
batch_size = 10
log_freq = int(len(train_random) / batch_size + 1)

# 创建基于随机拆分的PROTAC模型
protac_model_random = dc.models.MultitaskRegressor(
    n_tasks, 
    n_features, 
    layer_sizes, 
    dropouts=dropouts, 
    activation_fns=activation_fns,
    optimizer=optimizer, 
    batch_size=batch_size, 
    log_frequency=log_freq
)

# 创建基于骨架拆分的PROTAC模型
protac_model_scaffold = dc.models.MultitaskRegressor(
    n_tasks, 
    n_features, 
    layer_sizes, 
    dropouts=dropouts, 
    activation_fns=activation_fns,
    optimizer=optimizer, 
    batch_size=batch_size, 
    log_frequency=log_freq
)


现在让我们把所有内容结合起来实例化一个DeepChem模型吧！请注意，由于样本量较小，使用较小的批次大小实际上有助于提升性能。

In [ ]:
# 使用 TensorFlow 的方法计算可训练参数数量
param_count = protac_model_random.model.count_params()
print(f"There are {param_count} trainable parameters.")

# 查看模型的架构和详细信息
protac_model_random.model.summary()


接下来，定义验证函数，以防止过拟合。

In [ ]:
train_losses_random = []
val_losses_random = []

train_losses_scaffold = []
val_losses_scaffold = []

# 使用 deepchem 的 MSE 作为度量标准
metric = dc.metrics.Metric(dc.metrics.mean_squared_error)

n_epochs = 100
for i in range(n_epochs):
    # 训练随机拆分的模型，保存训练损失
    protac_model_random.fit(train_random, nb_epoch=1, all_losses=train_losses_random)

    # 训练骨架拆分的模型，保存训练损失
    protac_model_scaffold.fit(train_scaffold, nb_epoch=1, all_losses=train_losses_scaffold)

    # 每两个 epoch 进行一次验证
    if i % 2 == 0:
        # 在验证集上评估随机拆分模型，并保存验证损失
        loss_random = protac_model_random.evaluate(val_random, metrics=[metric])
        val_losses_random.append(loss_random['mean_squared_error'])

        # 在验证集上评估骨架拆分模型，并保存验证损失
        loss_scaffold = protac_model_scaffold.evaluate(val_scaffold, metrics=[metric])
        val_losses_scaffold.append(loss_scaffold['mean_squared_error'])

    print(f"Epoch {i+1}/{n_epochs} - Random Loss: {train_losses_random[-1]}, Scaffold Loss: {train_losses_scaffold[-1]}")


我们可以通过绘制记录的损失值，轻松查看训练过程的表现。

In [ ]:
# 计算训练步数和验证步数的频率
train_steps = [(i+1)*log_freq for i in range(len(train_losses_random))]  # 每个训练步骤的频率
val_steps = [(i+1)*(log_freq*2) for i in range(len(val_losses_random))]  # 每个验证步骤的频率

# 创建子图，设置图形大小为12x5
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

# 在第一个子图中绘制随机划分训练和验证集的损失曲线
ax[0].plot(train_steps, train_losses_random, label='Train loss')  # 绘制训练集损失曲线
ax[0].plot(val_steps, val_losses_random, label='Val loss')  # 绘制验证集损失曲线
ax[0].legend()  # 显示图例
ax[0].set_xlabel('Frequency of Steps')  # 设置x轴标签
ax[0].set_ylabel('Loss')  # 设置y轴标签
ax[0].set_title('Loss across train and validation random split')  # 设置图表标题

# 在第二个子图中绘制基于scaffold划分的训练和验证集的损失曲线
ax[1].plot(train_steps, train_losses_scaffold, label='Train loss')  # 绘制训练集损失曲线
ax[1].plot(val_steps, val_losses_scaffold, label='Val loss')  # 绘制验证集损失曲线
ax[1].legend()  # 显示图例
ax[1].set_xlabel('Frequency of Steps')  # 设置x轴标签
ax[1].set_ylabel('Loss')  # 设置y轴标签
ax[1].set_title('Loss across train and validation scaffold split')  # 设置图表标题

# 显示图形
plt.plot()


我们可以看到，模型在scaffold验证集上的表现较差，这是合理的，因为scaffold划分确保了更多的验证分子相对于训练分布是分布外的。

现在让我们对测试集进行一些推理，以评估我们的模型表现！

In [32]:
# 定义模型评估的度量标准，包括均方误差、皮尔逊相关系数和皮尔逊R²得分
metrics = [dc.metrics.Metric(dc.metrics.mean_squared_error),  # 均方误差
           dc.metrics.Metric(dc.metrics.pearsonr),  # 皮尔逊相关系数
           dc.metrics.Metric(dc.metrics.pearson_r2_score)]  # 皮尔逊R²得分

# 使用随机分割的模型对测试集进行评估，并计算上述度量标准
eval_metrics = protac_model_random.evaluate(test_random, metrics)

# 使用随机分割的模型对测试集进行预测，返回预测值
preds = protac_model_random.predict(test_random)


In [ ]:
# 遍历评估指标字典并打印每个指标的名称和值
for k, v in eval_metrics.items():
    print('{}: {}'.format(k, v))  # 输出每个评估指标的名称和值


In [ ]:
# 导入seaborn库，用于绘制可视化图表
# 如果还没安装seaborn 可以使用 pip install seaborn 进行安装
import seaborn as sns

# 将实际的log DC50值和预测的log DC50值合并成一个数组
preds_and_labels = np.concatenate((test_random.y.reshape([60, 1]),  # 实际值，调整为(60, 1)的形状
                                   preds.reshape([60, 1])),  # 预测值，调整为(60, 1)的形状
                                  axis=1)

# 将合并的数组转换为DataFrame，并为列名指定'实际log DC50值'和'预测log DC50值'
pred_df = pd.DataFrame(preds_and_labels, columns=['Actual log DC50 values', 'Predicted log DC50 values'])

# 使用seaborn绘制实际值与预测值的联合图
sns.jointplot(pred_df, x='Predicted log DC50 values', y='Actual log DC50 values')

# 在图中添加注释，显示皮尔逊相关系数R和R²得分
plt.annotate(f"R: {eval_metrics['pearsonr']:.2f}\nR²: {eval_metrics['pearson_r2_score']:.2f}",
             xy=(0.05, 0.95),  # 注释的位置（左上角）
             xycoords='axes fraction',  # 坐标系为图形区域
             ha='left',  # 水平对齐方式
             va='top',  # 垂直对齐方式
             fontsize=12,  # 字体大小
             bbox=dict(boxstyle='round,pad=0.5', edgecolor='black', facecolor='white'))  # 注释框样式

# 设置图表的标题
plt.suptitle('Predicted vs Actual log DC50 Values from Random Split')

# 调整布局，防止标题与图表重叠
plt.tight_layout()

# 显示图表
plt.show()


随机划分的结果看起来相当不错。现在让我们看看模型在 scaffold 划分上的表现如何。

In [ ]:
# 定义模型评估的度量标准，包括均方误差、皮尔逊相关系数和皮尔逊R²得分
metrics = [dc.metrics.Metric(dc.metrics.mean_squared_error),  # 均方误差
           dc.metrics.Metric(dc.metrics.pearsonr),  # 皮尔逊相关系数
           dc.metrics.Metric(dc.metrics.pearson_r2_score)]  # 皮尔逊R²得分

# 使用基于scaffold划分的模型对scaffold测试集进行评估，并计算上述度量标准
eval_metrics = protac_model_scaffold.evaluate(test_scaffold, metrics)

# 使用基于scaffold划分的模型对scaffold测试集进行预测，返回预测值
preds = protac_model_scaffold.predict(test_scaffold)

# 遍历评估指标字典并打印每个指标的名称和值
for k, v in eval_metrics.items():
    print('{}: {}'.format(k, v))  # 输出每个评估指标的名称和值


In [ ]:
# 导入seaborn库，用于绘制可视化图表
import seaborn as sns

# 将实际的log DC50值和预测的log DC50值合并成一个数组
preds_and_labels = np.concatenate((test_scaffold.y.reshape([60, 1]),  # 实际值，调整为(60, 1)的形状
                                   preds.reshape([60, 1])),  # 预测值，调整为(60, 1)的形状
                                  axis=1)

# 将合并的数组转换为DataFrame，并为列名指定'实际log DC50值'和'预测log DC50值'
pred_df = pd.DataFrame(preds_and_labels, columns=['Actual log DC50 values', 'Predicted log DC50 values'])

# 使用seaborn绘制实际值与预测值的联合图
sns.jointplot(pred_df, x='Predicted log DC50 values', y='Actual log DC50 values')

# 在图中添加注释，显示皮尔逊相关系数R和R²得分
plt.annotate(f"R: {eval_metrics['pearsonr']:.2f}\nR²: {eval_metrics['pearson_r2_score']:.2f}",
             xy=(0.05, 0.95),  # 注释的位置（左上角）
             xycoords='axes fraction',  # 坐标系为图形区域
             ha='left',  # 水平对齐方式
             va='top',  # 垂直对齐方式
             fontsize=12,  # 字体大小
             bbox=dict(boxstyle='round,pad=0.5', edgecolor='black', facecolor='white'))  # 注释框样式

# 设置图表的标题
plt.suptitle('Predicted vs Actual log DC50 Values from Scaffold Split')  # 设置标题为 "基于Scaffold划分的预测和实际log DC50值"

# 调整布局，防止标题与图表重叠
plt.tight_layout()

# 显示图表
plt.show()


模型在保留的 scaffold 测试集上表现显著较差，这是预期中的结果，因为模型相对简单。开发能够在分布外泛化的更复杂模型是许多研究领域的重点，从分子性质预测到计算机视觉再到自然语言处理都涉及这一点。总的来说，我希望本教程能成为对 PROTAC 世界的一个有用的介绍。请继续关注我们接下来的教程，我们将探讨如何思考 PROTAC 的设计！

阅读原文: https://deepchem.io/tutorials/introduction-to-protacs/